In [1]:
import tensorflow as tf
import pandas as pd


In [2]:
# constants
TEST_CSV_FILE_NAME = 'test_data.csv'
TRAIN_CSV_FILE_NAME = 'train_data.csv'
CSV_COLUMN_HEADERS = ['Value','Type','Result']
FEATURE_COLUMN_HEADERS = list(filter(lambda item: item != 'Result', CSV_COLUMN_HEADERS))
CATEGORIES = ['odd', 'even']


In [3]:
# read in the file with pandas and create data frames.
train_x_df = pd.read_csv(TRAIN_CSV_FILE_NAME,header=0)
test_x_df = pd.read_csv(TEST_CSV_FILE_NAME, header=0)

# vocab = train_x_df['Type'].unique()

print(test_x_df.head())


   Value           Type Result
0    4.0  <class 'int'>   even
1   33.0  <class 'int'>    odd
2  -63.0  <class 'int'>    odd
3  -79.0  <class 'int'>    odd
4  -88.0  <class 'int'>   even


In [4]:
# Need separate container for labels

train_y_df = train_x_df.pop('Result')
test_y_df = test_x_df.pop('Result')

print(train_y_df)


0         even
1          odd
2          odd
3         even
4          odd
          ... 
209080     odd
209081     odd
209082     odd
209083     odd
209084     odd
Name: Result, Length: 209085, dtype: object


In [5]:
# tf feature columns describing the inputs
vocab = [str(type(int)), str((type(6.9)))]
tf_feat_columns_array = [tf.feature_column.numeric_column(key='Value'), tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key='Type',vocabulary_list=vocab))]
# tf_feat_columns_array = [tf.feature_column.numeric_column(key='Value')]
print(tf_feat_columns_array)

[NumericColumn(key='Value', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Type', vocabulary_list=("<class 'type'>", "<class 'float'>"), dtype=tf.string, default_value=-1, num_oov_buckets=0))]


In [6]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=tf_feat_columns_array, 
    hidden_units=[100,100,100,100,100], 
    n_classes=len(CATEGORIES),
    label_vocabulary=CATEGORIES
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\JEREMY~1\\AppData\\Local\\Temp\\tmpdj7n2nca', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# Create input function

def test_input_func(features, labels, training:bool = True, batch_size:int = 25, num_epochs = 3):
    tf_dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if (training):
        tf_dataset.shuffle(10000)

    return tf_dataset.repeat(num_epochs).batch(batch_size)


In [8]:
classifier.train(lambda: test_input_func(train_x_df, train_y_df, training=True, num_epochs=3))

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\JEREMY~1\AppData\Local\Temp\tmpdj7n2nca\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.69100845, step = 0
INFO:tensorflow:global_step/sec: 102.433
INFO:tensorflow:loss = 4.8752275, step = 100 (0.977 sec)
INFO:tensorflow:global_step/sec: 123.636
INFO:tensorflow:loss = 9.566668, step = 200 (0.810 sec)

In [9]:
eval_result = classifier.evaluate(input_fn=lambda: test_input_func(test_x_df, test_y_df, training=False, num_epochs=1))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-01-20T11:04:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\JEREMY~1\AppData\Local\Temp\tmpdj7n2nca\model.ckpt-25091
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 8.80218s
INFO:tensorflow:Finished evaluation at 2023-01-20-11:04:45
INFO:tensorflow:Saving dict for global step 25091: accuracy = 0.6927902, accuracy_baseline = 0.6927902, auc = 0.5033647, auc_precision_recall = 0.31077993, average_loss = 11.022172, global_step = 25091, label/mean = 0.30720976, loss = 11.021703, precision = 0.0, prediction/mean = 0.009435685, recall = 0.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 25091: C:\Users\JEREMY~1\AppData\Local\Temp\tmpdj7n2nca\model.ckpt-25091


In [10]:
print(eval_result)

{'accuracy': 0.6927902, 'accuracy_baseline': 0.6927902, 'auc': 0.5033647, 'auc_precision_recall': 0.31077993, 'average_loss': 11.022172, 'label/mean': 0.30720976, 'loss': 11.021703, 'precision': 0.0, 'prediction/mean': 0.009435685, 'recall': 0.0, 'global_step': 25091}


In [11]:
print(f"The accuracy of the neural network is {eval_result['accuracy'] * 100}%")

The accuracy of the neural network is 69.27902102470398%


In [15]:
number_predict = -12312332
def predict_input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices((dict(features))).batch(batch_size)

FEATURE_COLUMN_HEADERS
predict = {}
predict['Value'] = [number_predict]
predict['Type'] = [str(type(number_predict))] # Tensorflow expects multiple predictions

predictions = classifier.predict(lambda: predict_input_fn(predict))

for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        CATEGORIES[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\JEREMY~1\AppData\Local\Temp\tmpdj7n2nca\model.ckpt-25091
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "odd" (100.0%)
